# I. Cralw data từ cafef.vn

## 1. Cài đặt thư viện 

In [1]:
pip install selenium 

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install xlsxwriter

Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
import pandas as pd

import os


## 2. Load web driver

Sử dụng Chrome

### 2.1. Load driver thủ công

In [ ]:
service = ChromeService(executable_path=r"C:\Web-driver\chromedriver.exe")
driver_manual = webdriver.Chrome(service=service)
driver_manual.get("https://s.cafef.vn/bao-cao-tai-chinh/hpg/bsheet/2023/4/0/0/bao-cao-tai-chinh-.chn")

### 2.2. Load driver bằng Driver Manager

In [11]:
service = ChromeService(ChromeDriverManager().install())
chrome_options = ChromeOptions()
chrome_options.add_argument("--headless=new")
driver_auto = webdriver.Chrome(service=service, options=chrome_options)
print(driver_auto.service.path)
driver_auto.get("https://s.cafef.vn/bao-cao-tai-chinh/dig/bsheet/2023/0/0/0/bao-cao-tai-chinh-.chn")

ReadTimeout: HTTPSConnectionPool(host='edgedl.me.gvt1.com', port=443): Read timed out. (read timeout=None)

## 3. Cralw dữ liệu

### Define các functions

In [2]:
def get_elements_value(items, convert_numeric=True):
    _ls = []
    if convert_numeric == False:
        for x in items:
            _ls.append(x.text)
    else:
        for x in items:
            num = x.text.strip().replace(',','')
            if num.isnumeric():
                _ls.append(int(num))
            else:
                _ls.append(num)
    return _ls

def year_col_process(driver,col): 
    x_path = f"//*[@id='tableContent']/tbody/tr/td[{col}]"
    rows = driver.find_elements(By.XPATH, x_path)
    return rows

def get_report_url(ticker, year, report_type):
    if report_type.upper() == 'BS':
        return f"https://s.cafef.vn/bao-cao-tai-chinh/{ticker}/bsheet/{year}/0/0/0/bao-cao-tai-chinh-.chn"
    if report_type.upper() == 'IS':
        return f"https://s.cafef.vn/bao-cao-tai-chinh/{ticker}/incsta/{year}/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn"
    if report_type.upper() == 'CF':
        return f"https://s.cafef.vn/bao-cao-tai-chinh/{ticker}/cashflow/{year}/0/0/0/luu-chuyen-tien-te-gian-tiep-.chn"

    return ""

def get_excel_data(driver, ticker, from_year, to_year, report_type):
    year = to_year 
    data = {}
    criteria_names = {}
    runable = True
    while(runable):
        url = get_report_url(ticker, year, report_type)
        print(url)
        driver.get(url)
        driver.implicitly_wait(5)
        
    
        if 'criteria' not in criteria_names:
            name_elements = year_col_process(driver, 1)
            criteria_names['criteria'] = get_elements_value(name_elements, False)
        
    
        year_elements = driver.find_elements(By.XPATH, "//*[@id='tblGridData']/tbody/tr/td")
        index_cols = {}
        i = 1
        for item in year_elements:
            str_item = item.text.strip()
            if str_item.isnumeric():
                index_cols[str_item] = i
            i += 1

        
        index_cols = dict(sorted(index_cols.items(), reverse=True))
    
        y = 0
        for key in index_cols:
            col = index_cols[key]
            items = year_col_process(driver, col)
            data[key] = get_elements_value(items)
            print(key)
            
            y = int(key)
            if y == from_year:
                runable = False
                break 
    
        if runable == True:
            year = y - 1
    
    
    data = dict(sorted(data.items()))  # Short dict
    data = criteria_names | data # Merge two dicts into one
    df = pd.DataFrame(data)
    
    save_as = f"{ticker}_{report_type}.xlsx"
    writer = pd.ExcelWriter(save_as,
                            engine= 'xlsxwriter',
                            engine_kwargs={'options': {'strings_to_numbers':True}})
    df.to_excel(writer, sheet_name=report_type, index=False)
    writer.close() 

    return 1

In [3]:
service = ChromeService(ChromeDriverManager().install())
chrome_options = ChromeOptions()
chrome_options.add_argument("--headless=new")
driver_auto = webdriver.Chrome(service=service, options=chrome_options)
print(driver_auto.service.path)

TICKER = 'DIG'
TO_YEAR = 2023
LOOK_BACK = 11
FROM_YEAR = TO_YEAR - LOOK_BACK + 1

report_types = ['BS', 'IS', 'CF']
for report in report_types:
    get_excel_data(driver_auto, TICKER, FROM_YEAR, TO_YEAR, report)
    
driver_auto.quit()
os.startfile(os.getcwd())

C:\Users\ASUS\.wdm\drivers\chromedriver\win64\119.0.6045.105\chromedriver-win32/chromedriver.exe
https://s.cafef.vn/bao-cao-tai-chinh/DIG/bsheet/2023/0/0/0/bao-cao-tai-chinh-.chn
2023
2022
2021
2020
https://s.cafef.vn/bao-cao-tai-chinh/DIG/bsheet/2019/0/0/0/bao-cao-tai-chinh-.chn
2019
2018
2017
2016
https://s.cafef.vn/bao-cao-tai-chinh/DIG/bsheet/2015/0/0/0/bao-cao-tai-chinh-.chn
2015
2014
2013
https://s.cafef.vn/bao-cao-tai-chinh/DIG/incsta/2023/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
2023
2022
2021
2020
https://s.cafef.vn/bao-cao-tai-chinh/DIG/incsta/2019/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
2019
2018
2017
2016
https://s.cafef.vn/bao-cao-tai-chinh/DIG/incsta/2015/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
2015
2014
2013
https://s.cafef.vn/bao-cao-tai-chinh/DIG/cashflow/2023/0/0/0/luu-chuyen-tien-te-gian-tiep-.chn
2023
2022
2021
2020
https://s.cafef.vn/bao-cao-tai-chinh/DIG/cashflow/2019/0/0/0/luu-chuyen-tien-te-gian-tiep-.chn
2019
2018
2017
2016
https://s.cafef.vn/bao-cao-tai-chinh/D

In [ ]:
# Sửa nhiều tệp
Ticker_list = ['HPG','DIG'] 
TO_YEAR = 2023
LOOK_BACK = 5
FROM_YEAR = TO_YEAR - LOOK_BACK + 1

report_types = ['BS', 'IS', 'CF']
for ticker in Ticker_list: 
    for report in report_types:
        get_excel_data(driver_auto, ticker, FROM_YEAR, TO_YEAR, report)
    
    driver_auto.quit()
    os.startfile(os.getcwd())